# Analyse pyBadlands stratigraphic output

If the stratigraphic structure is turned on in the input.xml file, **pyBadlands** produces sedimentary layers recorded by hdf5 files. The stratigraphic layers are defined on a regularly spaced grid and a layer is recorded at each layer time interval given by the user.

Here we show how we can visualise quickly the structure of the stratigraphic layer in an IPython notebook.

In [ ]:
import numpy as np
import cmocean as cmo
import matplotlib as mpl
from matplotlib import mlab, cm
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.colors as colors

# Import the python file (.py) which contains all defined functions
import stratalArchitecture as strata

# Display plots in SVG format
%config InlineBackend.figure_format = 'svg'

# Display plots in cells
%matplotlib inline

# 1-  Visualize stratigraphic layers on a cross-section

## 1.1- Loading the stratigraphic file

First we need to load the stratigraphic files. The files are located in the **h5/** folder in the simulation main output folder and are named using the following convention:
- `sed.time`T`.p`X`.hdf5`

with T the display time index and X the number of the partition (used in the parallel version). In cases where you ran your simulation in parallel you will also need to give the number of CPUs used (_cpus_).

To load a file you will need to give the folder path and the number of processors used in your simulation.

In [ ]:
# For more information regarding the function uncomment the following line.
# help(strata.stratalSection.__init__)

In [ ]:
folder = '/workspace/volume/case1/output/h5/'  # output folder path
strat = strata.stratalSection(folder, 1)

Then we need to load a particular output time interval (this is the T parameter in the hdf5 file name convention).

**Note**

This number is not always the number of sedimentary layers for this particular time step as you could have chosen in the input file to have more than 1 sedimentary layer recorded by output interval!

In [ ]:
# help(strat.loadStratigraphy)
# help(strat.loadTIN)

In [ ]:
timestep = 120  
strat.loadStratigraphy(timestep)  # load strata files
strat.loadTIN(timestep)  # load TIN files

**Important** 

If you want to change the timestep, you need to restart this script (in the top menu, Kernel->Restart) and run from the first cell.

## 1.2- Building a cross-section

To build a cross-section to visualise the stratigraphic layers, you will need to provide:

+ the coordinates of two points deliminating the cross-section **_(x1,y1)_** and **_(x2,y2)_**, 
+ the number of nodes that defines the resolution of this cross-section **_nbpts_** and
+ a gaussian filter value to smooth the the stratigraphic layer (**_gfilt_** a value of 0 can be used for non-smoothing).

Plotting the topography map from the model output can help you to define the location of the cross-section.

In [ ]:
# help(strat.plotSectionMap)

In [ ]:
strat.plotSectionMap(title='Topography map', xlegend='Distance (m)', ylegend='Distance (m)', 
                     color=cmo.cm.delta, crange=[-2000,2000], cs=None, size=(6,6))

In [ ]:
# Coordinates [x,y] of two points on the cross-section
cs=np.zeros((2,2))
cs[0,:] = [350000,350000]  # point 1
cs[1,:] = [350000,700000]  # point 2

# Interpolation parameters
nbpts = 700  
gfilt = 2  

# Show the location of the cross-section on the topography map
strat.plotSectionMap(title='Topography map', xlegend='Distance (m)', ylegend='Distance (m)',
                     color=cmo.cm.delta, colorcs='magenta', crange=[-2000,2000], cs=cs, size=(6,6))

In [ ]:
# Build cross-section
strat.buildSection(xo = cs[0,0], yo = cs[0,1], xm = cs[1,0], ym = cs[1,1], pts = nbpts, gfilter = gfilt)

## 1.3- Visualize stratal stacking pattern coloured by time

First, we use **plotly** to visualise the vertival cross-section of stratal stacking pattern.

In [ ]:
strata.viewSection(width = 800, height = 500, cs = strat, 
            dnlay = 2, rangeX=[2000, 10000], rangeY=[-400,200],
            linesize = 0.5, title='Stratal stacking pattern coloured by time')

## 1.4- Visualize stratal stacking pattern coloured by facies

First we build paleo-depositional environment (facies) structure based on the paleo-water depth. For example ([reference](https://opentextbc.ca/geology/chapter/6-3-depositional-environments-and-sedimentary-basins/)),

<img src="../images/depo-envi.png" alt="depositional environments" width="800" height="500"/>

In [ ]:
def buildEnviID_facies(cs = None, depthIDs = None):
    envi = np.zeros((cs.nz, len(depthIDs)+2))
    
    for i in range(cs.nz):
        envi[i][0] = np.amax(np.where(cs.secElev[i] > 50)) # onlap
        for j in range(len(depthIDs)):
            envi[i][j+1] = np.amax(np.where((cs.secElev[i]) >= depthIDs[j])[0])
        envi[i][len(depthIDs)+1] = np.amax(np.where((cs.secTh[i]) >= 0.01)[0]) # downlap
    #
    return envi

In [ ]:
# Specify the range of water depth for the depositional environments, see the table above
depthID = [0, -30, -100, -300, -500]

# Build an array of depositional environment ID (enviID)
enviID = np.zeros((strat.nz, len(depthID)+2))
enviID = buildEnviID_facies(cs = strat, depthIDs = depthID)

In [ ]:
# Plot stratal stacking pattern colored by paleo-depositional environments
fig = plt.figure(figsize = (7,7))
plt.rc("font", size=12)
# 
ax = fig.add_axes([0.11, 0.57, 0.85, 0.4])
p = 0
layID = []
xi00 = np.array(strat.dist)/1000.  # change the unit to be be km
colors = ['limegreen','darkkhaki','sandybrown','khaki','c','teal']
for i in range(0,strat.nz+1,3):
    if i == strat.nz:
        i=strat.nz-1
    layID.append(i)
    ax.plot(xi00,strat.secDep[layID[p]],'-',color='k',linewidth=0.1)
    if len(layID)>1:
        for j in range(len(depthID)+1):
            ID1=enviID[i][j]
            ID2=enviID[i][j+1]
            for k in range(int(ID1),int(ID2)):
                ax.fill_between([xi00[k],xi00[k+1]], [strat.secDep[layID[p-1]][k], strat.secDep[layID[p-1]][k+1]],
                                max(strat.secDep[layID[p-1]]), color=colors[j])
    if (max(strat.secDep[layID[p]]) <= max(strat.secDep[layID[p-1]])):
        plt.fill_between(xi00, strat.secDep[layID[p]], max(strat.secDep[layID[p-1]]), color='white')
    else:
        plt.fill_between(xi00, strat.secDep[layID[p]], max(strat.secDep[layID[p]]), color='white')
    p=p+1
#     
ax.plot(xi00,strat.secDep[strat.nz-1],'-',color='k',linewidth=0.3)  # top line
ax.plot(xi00,strat.secDep[0],'-',color='k',linewidth=0.3)  # bottom line
# ax.set_xlim([340, 200])
# ax.set_ylim([-1000, 350])
plt.xlabel('Distance (km)',fontsize=12)
plt.ylabel('Elevation (m)',fontsize=12)

# Save the plot
# fig.savefig("Strata.jpg", dpi=300)

# 2-  Build a Wheeler diagram

Wheeler diagram (or chronostratigraphic chart) is a powerful tool to document unconformities between sequences, and to understand the evolution of sedimentary stacking patterns and their relationships to sea level. It displays the horizontal distribution of contemporaneous sedimentary layer sequences, as well as hiatuses in sedimentation.

In [ ]:
def buildEnviID_WD(cs = None, depthID = None):
    enviID = np.zeros((cs.nz, len(cs.dist)))
    
    # Build a 2D array of the depositional environment ID (enviID) 
    for j in range(cs.nz):
        for i in range(len(cs.dist)):
            if (cs.secElev[j][i]) > (depthID[0]):
                enviID[j][i] = 0
            elif (cs.secElev[j][i]) > (depthID[1]):
                enviID[j][i] = 1
            elif (cs.secElev[j][i]) > (depthID[2]):
                enviID[j][i] = 2
            elif (cs.secElev[j][i]) > (depthID[3]):
                enviID[j][i] = 3
            elif (cs.secElev[j][i]) > (depthID[4]):
                enviID[j][i] = 4
            else:
                enviID[j][i] = 5
    
    # Where the deposited thickness is less than 0.5 m, the enviID will be set to -1 (i.e. will be coloured in white).
    # You can change the value of '0.5'.
    for j in range(cs.nz):
        for i in range(len(cs.dist)):
            if (cs.secTh[j][i]) <= 0.5:
                enviID[j][i] = -1
    
    return enviID

In [ ]:
# Define colors for depositional environments, with number of colors equals to len(depthID) + 2
colorDepoenvi = ['white','limegreen','darkkhaki','sandybrown','khaki','c','teal'] 
# 'White' colors where either no deposition or deposited sediemnt thickness < 0.5 m.

# Build an array of depositional environment ID (enviID_WD)
enviID_WD = np.zeros((strat.nz, len(strat.dist)))
enviID_WD = buildEnviID_WD(cs = strat, depthID = depthID)

# Time structure of the model, corresponding to the Time structure in the input.xml file
start_time = 0.  # the start time of the model run [a]
disptime = 100000.  # the layer interval of the strata module [a]
end_time = start_time + disptime * timestep  # the time of the loaded output [a]
layertime = np.linspace(start_time,end_time,strat.nz)  # time of the layers

In [ ]:
# Plot Wheeler diagram
fig = plt.figure(figsize = (7,4))
plt.rc("font", size=13)
    
ax = fig.add_axes([0.15, 0.18, 0.74, 0.76])
# Define the meshgrid of the Wheeler diagram, in which X axis is distance along the cross-section and Y axis is time
dist = strat.dist  
Dist, Time = np.meshgrid(dist, layertime)

cmap = colors.ListedColormap(colorDepoenvi)
boundaries = [-1, 6]
levels=[-1.5,-0.5,0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm = colors.Normalize(boundaries, cmap.N)
plt.imshow(enviID_WD[:,:], interpolation='nearest', cmap=cmap, origin='lower', 
           extent=[dist.min()/1000, dist.max()/1000, layertime.min()/1e6, layertime.max()/1e6], 
           aspect='auto', vmax=5.5, vmin=-1.5)
# plt.colorbar()

# Plot the horizontal time lines with the same interval with the stratal stacking pattern
for i in range(0,strat.nz,3): 
    plt.axhline(layertime[i]/1e6, color='k', linewidth=0.2)

plt.xlabel('Distance (km)')
plt.ylabel('Time (Myr)')
# 
# fig.savefig("WheeDiag.jpg", dpi=300)

# 3-  Extract synthetic cores

To plot the synthetic cores (vertical stacking patterns) at any locations on the cross-section, you need to give the location of the core.

In [ ]:
# Location of the core on the cross-section (m)
posit = 150000

# Plot the core
strata.viewCore(width = 2, height = 5, cs = strat, enviID = enviID, posit = posit, time = layertime, 
                color = colorDepoenvi, rangeX = None, rangeY = None, savefig = 'Yes', figname = 'delta_core')